### Importar librerías y cargar datos

In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Nombre del archivo CSV con múltiples hojas
archivo_csv = "data/Base_2019.xlsx"

# Leer cada hoja del archivo CSV en un diccionario de DataFrames
diccionario_dataframes = pd.read_excel(archivo_csv, sheet_name=None)

# Iterar a través de las hojas y procesar cada DataFrame
for nombre_hoja, dataframe in diccionario_dataframes.items():
    # Imprimir el nombre de la hoja
    print("Hoja:", nombre_hoja)
    # Imprimir el contenido del DataFrame
    print(dataframe.head())
    print("---------------------------------------------------------------------------------")

Hoja: ACCIDENTES
   idFormulario      Dia      Fecha MES_PROCESADO   Oficina  GravedadCod  \
0             1    LUNES 2019-01-21         ENERO  11001000            2   
1             2   MARTES 2019-01-08         ENERO  11001000            3   
2             3   MARTES 2019-01-01         ENERO  11001000            2   
3             4   MARTES 2019-01-01         ENERO  11001000            3   
4             5  VIERNES 2019-01-18         ENERO  11001000            3   

  GravedadNombre  ClaseCodigo ClaseNombre  ChoqueCodigo  ... CON_CARGA  \
0    Con Heridos            2   Atropello           NaN  ...        NO   
1     Solo Daños            1      Choque           1.0  ...        NO   
2    Con Heridos            1      Choque           1.0  ...        NO   
3     Solo Daños            1      Choque           1.0  ...        NO   
4     Solo Daños            1      Choque           1.0  ...        NO   

   CON_EMBRIAGUEZ CON_HUECOS  CON_MENORES CON_MOTO  CON_PEATON  \
0              

### Selección preliminar de variables de interés

In [3]:
columns_deleted = [
    "ClaseOficial",
    "GradoOficial",
    "UnidadOficial",
    "FechaExpedicion",
    "EstabaServicioOficial",
    "Vehiculo",
    "RadioAccion",
    "Fecha",
    "MES_PROCESADO",
    "DIA_PROCESADO",
    "OficinaExpedicionLicencia",
    "EsPropietarioVehiculo",
    "ModalidadVehiculo"
]

columns_to_consider = [
    "IdConductor",
    "idFormulario",
    "EDAD_PROCESADA",
    "LLevaCinturon",
    "LLevaChaleco",
    "LLevaCasco",
    "Sexo",
    "GRAVEDAD_PROCESADA",
    "PortaLicencia",
    "CodigoCategoriaLicencia",
    "CodigoRestriccionLicencia",
    "ModeloVehiculo",
    "CapacidadCarga",
    "CantidadPasajeros",
    "ClaseVehiculo",
    "VEHICULO_VIAJABA_CLASIFICADO",
    "ServicioVehiculo",
    "PosseSeguroResponsabilidad",
    "TipoFalla",
    "CON_BICICLETA",
    "CON_CARGA",
    "CON_EMBRIAGUEZ",
    "CON_HUECOS",
    "CON_MENORES",
    "CON_MOTO",
    "CON_PEATON",
    "CON_PERSONA_MAYOR",
    "CON_RUTAS",
    "CON_TPI",
    "CON_VELOCIDAD"
]

def create_conductor_id(row):
    return str(row['idFormulario']) + '-' + str(row['Vehiculo'])

CONDUCTORES = diccionario_dataframes['CONDUCTORES']
CONDUCTORES['IdConductor'] = CONDUCTORES.apply(create_conductor_id , axis=1)
CONDUCTORES = CONDUCTORES[columns_to_consider]
ACCIDENTES = diccionario_dataframes["ACCIDENTES"]
VICTIMAS= diccionario_dataframes["VICTIMAS"]

CONDUCTORES.head()

,IdConductor,idFormulario,EDAD_PROCESADA,LLevaCinturon,LLevaChaleco,LLevaCasco,Sexo,GRAVEDAD_PROCESADA,PortaLicencia,CodigoCategoriaLicencia,...,CON_CARGA,CON_EMBRIAGUEZ,CON_HUECOS,CON_MENORES,CON_MOTO,CON_PEATON,CON_PERSONA_MAYOR,CON_RUTAS,CON_TPI,CON_VELOCIDAD
0,1-1,1,SIN INFORMACION,NaN,NaN,NaN,NO APLICA,ILESA,NaN,NaN,...,NO,NO,NO,NO,NO,SI,SI,NO,NO,NO
1,2-1,2,23,NaN,N,N,MASCULINO,ILESA,S,A2,...,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO
2,2-2,2,SIN INFORMACION,NaN,NaN,NaN,NO APLICA,ILESA,NaN,NaN,...,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO
3,3-2,3,26,NaN,N,N,MASCULINO,HERIDO VALORADO,N,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
4,3-1,3,28,N,NaN,NaN,MASCULINO,ILESA,S,C3,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


### Preprocesar los Datos (Tratar Vacíos y Outliers Extremos) de Conductores

Se imputan los datos vacíos de las columnas, en su mayoría por 'SIN INFORMACION' y en algunos casos por la clase que por lógica según los datos, deberían pertenecer al estar vacíos (que es el caso de Modelo ,Modalidad Vehículo y el caso de TipoFalla):

In [4]:
replacement_value = 'SIN INFORMACION'
modalidad_vehiculo_replacement_value = 'Particular'
tipo_falla_replacement_value = 'No Aplica'
modelo_vehiculo_replacement_value = int(np.mean(CONDUCTORES['ModeloVehiculo'].dropna().values))
edad_promedio = int(CONDUCTORES[CONDUCTORES['EDAD_PROCESADA'] != 'SIN INFORMACION']['EDAD_PROCESADA'].astype(int).mean())

CONDUCTORES.loc[:, 'LLevaChaleco'] = CONDUCTORES['LLevaChaleco'].fillna(replacement_value)
CONDUCTORES.loc[:, 'LLevaCinturon']= CONDUCTORES['LLevaCinturon'].fillna(replacement_value)
CONDUCTORES.loc[:, 'LLevaCasco'] = CONDUCTORES['LLevaCasco'].fillna(replacement_value)
CONDUCTORES.loc[:, 'PortaLicencia'] = CONDUCTORES['PortaLicencia'].fillna(replacement_value)
CONDUCTORES.loc[:, 'CodigoCategoriaLicencia'] = CONDUCTORES['CodigoCategoriaLicencia'].fillna(replacement_value)
CONDUCTORES.loc[:, 'CodigoRestriccionLicencia'] = CONDUCTORES['CodigoRestriccionLicencia'].fillna(replacement_value)
CONDUCTORES.loc[:, 'ClaseVehiculo'] = CONDUCTORES['ClaseVehiculo'].fillna(replacement_value)
CONDUCTORES.loc[:, 'ServicioVehiculo'] = CONDUCTORES['ServicioVehiculo'].fillna(replacement_value)
CONDUCTORES.loc[:, 'TipoFalla'] = CONDUCTORES['TipoFalla'].fillna(tipo_falla_replacement_value)
CONDUCTORES.loc[:, 'ModeloVehiculo'] = CONDUCTORES['ModeloVehiculo'].fillna(modelo_vehiculo_replacement_value).astype(int)
CONDUCTORES.loc[:, 'Edad'] = CONDUCTORES['EDAD_PROCESADA'].replace({'SIN INFORMACION': edad_promedio})
CONDUCTORES.loc[:, 'Gravedad'] = CONDUCTORES['GRAVEDAD_PROCESADA']
CONDUCTORES.loc[:, 'VehiculoViajabaClasificado'] = CONDUCTORES['VEHICULO_VIAJABA_CLASIFICADO']

CONDUCTORES = CONDUCTORES.drop('EDAD_PROCESADA', axis=1)
CONDUCTORES = CONDUCTORES.drop('GRAVEDAD_PROCESADA', axis=1)
CONDUCTORES = CONDUCTORES.drop('VEHICULO_VIAJABA_CLASIFICADO', axis=1)

CONDUCTORES['LLevaChaleco'] = CONDUCTORES['LLevaChaleco'].apply(lambda x: 1 if x == 'S' else -1 if x == 'N' else 0)
CONDUCTORES['LLevaCinturon'] = CONDUCTORES['LLevaCinturon'].apply(lambda x: 1 if x == 'S' else -1 if x == 'N' else 0)
CONDUCTORES['LLevaCasco'] = CONDUCTORES['LLevaCasco'].apply(lambda x: 1 if x == 'S' else -1 if x == 'N' else 0)

CONDUCTORES['HuboFallaVehiculo'] = CONDUCTORES['TipoFalla'].apply(lambda x: 0 if x == 'No Aplica' or x == '' else 1)

CONDUCTORES = CONDUCTORES.drop('TipoFalla', axis=1)

def calculate_lleva(row):
    sumatoria = row['LLevaChaleco'] + row['LLevaCinturon'] + row['LLevaCasco']
    if sumatoria > 0:
        return 1
    elif sumatoria < 0:
        return -1
    else:
        return 0

CONDUCTORES['CumpleNormasSeguridad'] = CONDUCTORES.apply(calculate_lleva, axis=1)
CONDUCTORES = CONDUCTORES.drop('LLevaChaleco', axis=1)
CONDUCTORES = CONDUCTORES.drop('LLevaCinturon', axis=1)
CONDUCTORES = CONDUCTORES.drop('LLevaCasco', axis=1)


C:\Users\dhoyoso\AppData\Local\Temp\ipykernel_26028\1867276894.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CONDUCTORES.loc[:, 'Edad'] = CONDUCTORES['EDAD_PROCESADA'].replace({'SIN INFORMACION': edad_promedio})
C:\Users\dhoyoso\AppData\Local\Temp\ipykernel_26028\1867276894.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CONDUCTORES.loc[:, 'Gravedad'] = CONDUCTORES['GRAVEDAD_PROCESADA']
C:\Users\dhoyoso\AppData\Local\Temp\ipykernel_26028\1867276894.py:19: SettingWithCopyWarning: 
A value is tryi

In [5]:
def count_victims(df):
    """
    Cuenta el número de víctimas por `idFormulario`.

    Args:
    df: El DataFrame de víctimas.

    Returns:
    Un diccionario que mapea el `idFormulario` a la cuenta de víctimas.
    """

    id_formularios = df['idFormulario'].unique()
    victim_count = {}
    for id_formulario in id_formularios:
        victim_count[id_formulario] = 0
    for index, row in df.iterrows():
        id_formulario = row['idFormulario']
        victim_count[id_formulario] += 1
    return victim_count

In [6]:
# Creamos el DataFrame 'victim_count'

# Contamos las víctimas
victim_count = count_victims(VICTIMAS)
len(victim_count)

7423

In [7]:
# Identifica los valores en 'idFormulario' de CONDUCTORES que no están en el diccionario
missing_values = CONDUCTORES[~CONDUCTORES['idFormulario'].isin(victim_count.keys())]['idFormulario']

# Imprime los valores faltantes
print("Valores en 'idFormulario' de CONDUCTORES que no están en el diccionario:")
print(len(missing_values))

Valores en 'idFormulario' de CONDUCTORES que no están en el diccionario:
55780


In [8]:
# Crea una lista de los ID de formularios de conductores
id_formularios_conductores = CONDUCTORES['idFormulario'].unique()

# Filtra VICTIMAS para que solo contenga las filas con ID de formularios que existen en CONDUCTORES
victimas_filtradas = VICTIMAS[VICTIMAS['idFormulario'].isin(id_formularios_conductores)]

# Agrupa VICTIMAS por 'idFormulario' y cuenta la cantidad de víctimas en cada formulario
cuenta_victimas = victimas_filtradas.groupby('idFormulario').size().reset_index(name='CantidadVictimas')

# Realiza la fusión solo con los ID de formularios de conductores
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_victimas, on='idFormulario', how='left')

# Rellenamos los valores faltantes con 0
CONDUCTORES['CantidadVictimas'] = CONDUCTORES['CantidadVictimas'].fillna(0)
CONDUCTORES

,IdConductor,idFormulario,Sexo,PortaLicencia,CodigoCategoriaLicencia,CodigoRestriccionLicencia,ModeloVehiculo,CapacidadCarga,CantidadPasajeros,ClaseVehiculo,...,CON_PERSONA_MAYOR,CON_RUTAS,CON_TPI,CON_VELOCIDAD,Edad,Gravedad,VehiculoViajabaClasificado,HuboFallaVehiculo,CumpleNormasSeguridad,CantidadVictimas
0,1-1,1,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,SI,NO,NO,NO,38,ILESA,SIN INFORMACION,0,0,1.0
1,2-1,2,MASCULINO,S,A2,NINGUNA,2017.0,0.00,2,Motocicleta,...,SI,NO,NO,NO,23,ILESA,MOTOCICLETA,0,-1,0.0
2,2-2,2,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,SI,NO,NO,NO,38,ILESA,SIN INFORMACION,0,0,0.0
3,3-2,3,MASCULINO,N,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,Bicicleta,...,NO,NO,NO,NO,26,HERIDO VALORADO,BICICLETA,0,-1,0.0
4,3-1,3,MASCULINO,S,C3,NINGUNA,2014.0,0.00,53,Bus,...,NO,NO,NO,NO,28,ILESA,TRANSPORTE DE PASAJEROS,0,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66174,34988-2,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,7,Campero,...,NO,NO,NO,NO,41,ILESA,LIVIANO,0,1,0.0
66175,34988-1,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,5,Automovil,...,NO,NO,NO,NO,25,ILESA,LIVIANO,0,1,0.0
66176,34989-2,34989,MASCULINO,S,B2,NINGUNA,2019.0,0.00,5,Automovil,...,NO,NO,NO,NO,38,ILESA,LIVIANO,0,1,0.0
66177,34989-1,34989,MASCULINO,S,C2,NINGUNA,2014.0,84.00,0,"Camion, Furgon",...,NO,NO,NO,NO,21,ILESA,CARGA,0,1,0.0


In [9]:
# Crear tres DataFrames para cada gravedad
victimas_gravedad1 = VICTIMAS[VICTIMAS['GRAVEDAD_PROCESADA'] == 'HERIDO VALORADO']
victimas_gravedad2 = VICTIMAS[VICTIMAS['GRAVEDAD_PROCESADA'] == 'HERIDO HOSPITALIZADO']
victimas_gravedad3 = VICTIMAS[VICTIMAS['GRAVEDAD_PROCESADA'] == 'MUERTA']

# Calcular la cantidad de veces que aparece cada gravedad en cada formulario de CONDUCTORES
cuenta_gravedad1 = victimas_gravedad1.groupby('idFormulario').size().reset_index(name='Cantidad_HERIDO_VALORADO')
cuenta_gravedad2 = victimas_gravedad2.groupby('idFormulario').size().reset_index(name='Cantidad_HERIDO_HOSPITALIZADO')
cuenta_gravedad3 = victimas_gravedad3.groupby('idFormulario').size().reset_index(name='Cantidad_MUERTA')

# Fusionar las columnas de cantidad de gravedades en CONDUCTORES
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_gravedad1, on='idFormulario', how='left')
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_gravedad2, on='idFormulario', how='left')
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_gravedad3, on='idFormulario', how='left')

# Rellenar NaN con 0 en caso de que no haya víctimas de alguna gravedad en un formulario
CONDUCTORES.fillna(0, inplace=True)
CONDUCTORES

,IdConductor,idFormulario,Sexo,PortaLicencia,CodigoCategoriaLicencia,CodigoRestriccionLicencia,ModeloVehiculo,CapacidadCarga,CantidadPasajeros,ClaseVehiculo,...,CON_VELOCIDAD,Edad,Gravedad,VehiculoViajabaClasificado,HuboFallaVehiculo,CumpleNormasSeguridad,CantidadVictimas,Cantidad_HERIDO_VALORADO,Cantidad_HERIDO_HOSPITALIZADO,Cantidad_MUERTA
0,1-1,1,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,NO,38,ILESA,SIN INFORMACION,0,0,1.0,1.0,0.0,0.0
1,2-1,2,MASCULINO,S,A2,NINGUNA,2017.0,0.00,2,Motocicleta,...,NO,23,ILESA,MOTOCICLETA,0,-1,0.0,0.0,0.0,0.0
2,2-2,2,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,NO,38,ILESA,SIN INFORMACION,0,0,0.0,0.0,0.0,0.0
3,3-2,3,MASCULINO,N,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,Bicicleta,...,NO,26,HERIDO VALORADO,BICICLETA,0,-1,0.0,0.0,0.0,0.0
4,3-1,3,MASCULINO,S,C3,NINGUNA,2014.0,0.00,53,Bus,...,NO,28,ILESA,TRANSPORTE DE PASAJEROS,0,-1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66174,34988-2,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,7,Campero,...,NO,41,ILESA,LIVIANO,0,1,0.0,0.0,0.0,0.0
66175,34988-1,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,5,Automovil,...,NO,25,ILESA,LIVIANO,0,1,0.0,0.0,0.0,0.0
66176,34989-2,34989,MASCULINO,S,B2,NINGUNA,2019.0,0.00,5,Automovil,...,NO,38,ILESA,LIVIANO,0,1,0.0,0.0,0.0,0.0
66177,34989-1,34989,MASCULINO,S,C2,NINGUNA,2014.0,84.00,0,"Camion, Furgon",...,NO,21,ILESA,CARGA,0,1,0.0,0.0,0.0,0.0


In [10]:
# Crear tres DataFrames para cada clasificacion de victima
victimas_peaton = VICTIMAS[VICTIMAS['Peaton_Pasajero'] == 'Peaton']
victimas_acompanante = VICTIMAS[VICTIMAS['Peaton_Pasajero'] == 'Acompañante']
victimas_pasajero = VICTIMAS[VICTIMAS['Peaton_Pasajero'] == 'Pasajero']

# Calcular la cantidad de veces que aparece cada clasificación de victima en cada formulario de CONDUCTORES
cuenta_peaton = victimas_peaton.groupby('idFormulario').size().reset_index(name='Cantidad_PEATON')
cuenta_acompanante = victimas_acompanante.groupby('idFormulario').size().reset_index(name='Cantidad_ACOMPANANTE')
cuenta_pasajero = victimas_pasajero.groupby('idFormulario').size().reset_index(name='Cantidad_PASAJERO')

# Fusionar las columnas de cantidad de clasificaciones de victimas en CONDUCTORES
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_peaton, on='idFormulario', how='left')
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_acompanante, on='idFormulario', how='left')
CONDUCTORES = pd.merge(CONDUCTORES, cuenta_pasajero, on='idFormulario', how='left')

# Rellenar NaN con 0 en caso de que no haya víctimas con alguna clasificación en un formulario
CONDUCTORES.fillna(0, inplace=True)
CONDUCTORES

,IdConductor,idFormulario,Sexo,PortaLicencia,CodigoCategoriaLicencia,CodigoRestriccionLicencia,ModeloVehiculo,CapacidadCarga,CantidadPasajeros,ClaseVehiculo,...,VehiculoViajabaClasificado,HuboFallaVehiculo,CumpleNormasSeguridad,CantidadVictimas,Cantidad_HERIDO_VALORADO,Cantidad_HERIDO_HOSPITALIZADO,Cantidad_MUERTA,Cantidad_PEATON,Cantidad_ACOMPANANTE,Cantidad_PASAJERO
0,1-1,1,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,SIN INFORMACION,0,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2-1,2,MASCULINO,S,A2,NINGUNA,2017.0,0.00,2,Motocicleta,...,MOTOCICLETA,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2-2,2,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,SIN INFORMACION,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3-2,3,MASCULINO,N,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,Bicicleta,...,BICICLETA,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3-1,3,MASCULINO,S,C3,NINGUNA,2014.0,0.00,53,Bus,...,TRANSPORTE DE PASAJEROS,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66174,34988-2,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,7,Campero,...,LIVIANO,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66175,34988-1,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,5,Automovil,...,LIVIANO,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66176,34989-2,34989,MASCULINO,S,B2,NINGUNA,2019.0,0.00,5,Automovil,...,LIVIANO,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66177,34989-1,34989,MASCULINO,S,C2,NINGUNA,2014.0,84.00,0,"Camion, Furgon",...,CARGA,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Fusionar las columna de gravedadCod de ACCIDENTES en CONDUCTORES
CONDUCTORES = pd.merge(CONDUCTORES, ACCIDENTES[['idFormulario', 'GravedadCod']], on='idFormulario', how='left')
CONDUCTORES['GravedadAccidente'] = CONDUCTORES['GravedadCod']
CONDUCTORES = CONDUCTORES.drop('GravedadCod', axis=1)

# Rellenar NaN con 0 en caso de que no haya accidentes con alguna gravedad en un formulario
CONDUCTORES.fillna(0, inplace=True)
CONDUCTORES

,IdConductor,idFormulario,Sexo,PortaLicencia,CodigoCategoriaLicencia,CodigoRestriccionLicencia,ModeloVehiculo,CapacidadCarga,CantidadPasajeros,ClaseVehiculo,...,HuboFallaVehiculo,CumpleNormasSeguridad,CantidadVictimas,Cantidad_HERIDO_VALORADO,Cantidad_HERIDO_HOSPITALIZADO,Cantidad_MUERTA,Cantidad_PEATON,Cantidad_ACOMPANANTE,Cantidad_PASAJERO,GravedadAccidente
0,1-1,1,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,0,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2
1,2-1,2,MASCULINO,S,A2,NINGUNA,2017.0,0.00,2,Motocicleta,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,2-2,2,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,3-2,3,MASCULINO,N,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,Bicicleta,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,3-1,3,MASCULINO,S,C3,NINGUNA,2014.0,0.00,53,Bus,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66174,34988-2,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,7,Campero,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66175,34988-1,34988,MASCULINO,S,B1,NINGUNA,2009.0,0.00,5,Automovil,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66176,34989-2,34989,MASCULINO,S,B2,NINGUNA,2019.0,0.00,5,Automovil,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66177,34989-1,34989,MASCULINO,S,C2,NINGUNA,2014.0,84.00,0,"Camion, Furgon",...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [12]:
CONDUCTORES = CONDUCTORES.drop('idFormulario', axis = 1)
CONDUCTORES

,IdConductor,Sexo,PortaLicencia,CodigoCategoriaLicencia,CodigoRestriccionLicencia,ModeloVehiculo,CapacidadCarga,CantidadPasajeros,ClaseVehiculo,ServicioVehiculo,...,HuboFallaVehiculo,CumpleNormasSeguridad,CantidadVictimas,Cantidad_HERIDO_VALORADO,Cantidad_HERIDO_HOSPITALIZADO,Cantidad_MUERTA,Cantidad_PEATON,Cantidad_ACOMPANANTE,Cantidad_PASAJERO,GravedadAccidente
0,1-1,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,SIN INFORMACION,...,0,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2
1,2-1,MASCULINO,S,A2,NINGUNA,2017.0,0.00,2,Motocicleta,Particular,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,2-2,NO APLICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,SIN INFORMACION,SIN INFORMACION,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,3-2,MASCULINO,N,SIN INFORMACION,SIN INFORMACION,2011.0,0.00,0,Bicicleta,SIN INFORMACION,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,3-1,MASCULINO,S,C3,NINGUNA,2014.0,0.00,53,Bus,Publico,...,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66174,34988-2,MASCULINO,S,B1,NINGUNA,2009.0,0.00,7,Campero,Particular,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66175,34988-1,MASCULINO,S,B1,NINGUNA,2009.0,0.00,5,Automovil,Particular,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66176,34989-2,MASCULINO,S,B2,NINGUNA,2019.0,0.00,5,Automovil,Particular,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
66177,34989-1,MASCULINO,S,C2,NINGUNA,2014.0,84.00,0,"Camion, Furgon",Publico,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [13]:
CONDUCTORES.columns

Index(['IdConductor', 'Sexo', 'PortaLicencia', 'CodigoCategoriaLicencia',
       'CodigoRestriccionLicencia', 'ModeloVehiculo', 'CapacidadCarga',
       'CantidadPasajeros', 'ClaseVehiculo', 'ServicioVehiculo',
       'PosseSeguroResponsabilidad', 'CON_BICICLETA', 'CON_CARGA',
       'CON_EMBRIAGUEZ', 'CON_HUECOS', 'CON_MENORES', 'CON_MOTO', 'CON_PEATON',
       'CON_PERSONA_MAYOR', 'CON_RUTAS', 'CON_TPI', 'CON_VELOCIDAD', 'Edad',
       'Gravedad', 'VehiculoViajabaClasificado', 'HuboFallaVehiculo',
       'CumpleNormasSeguridad', 'CantidadVictimas', 'Cantidad_HERIDO_VALORADO',
       'Cantidad_HERIDO_HOSPITALIZADO', 'Cantidad_MUERTA', 'Cantidad_PEATON',
       'Cantidad_ACOMPANANTE', 'Cantidad_PASAJERO', 'GravedadAccidente'],
      dtype='object')

In [14]:
def describe(df, nombre):
    print('******************************************************************************************')
    print(' *************************************  {}  **************************************'.format(nombre))
    print('******************************************************************************************')
    print()
    print('--> La dimensión de los {} es de {}'.format(nombre, df.shape))
    print()
    print('--> Las columnas de {} son: \n \n{} '.format(nombre, list(df.columns)))
    print()
    print('--> Información del Dataframe de {}:'.format(nombre))
    print()
    df.info()
    print()
    print("--> Registros vacíos por columna:")
    print()
    print(df.isnull().sum())
    print()
    print('--> Estadísticas descriptivas por columnas numericas:')
    print()
    print(df.describe())
    print()
    print('--> Estadísticas descriptivas por columnas categóricas:')
    print()
    print(df.describe(include=['object'])[list(set(df.columns) - set(df.select_dtypes(include=['number', 'datetime']).columns))[0:15]])
    print()
    print(df.describe(include=['object'])[list(set(df.columns) - set(df.select_dtypes(include=['number', 'datetime']).columns))[15:30]])
    print()
    print(df.describe(include=['object'])[list(set(df.columns) - set(df.select_dtypes(include=['number', 'datetime']).columns))[30:]])
    print()
    print()

describe(CONDUCTORES,'CONDUCTORES')

******************************************************************************************
 *************************************  CONDUCTORES  **************************************
******************************************************************************************

--> La dimensión de los CONDUCTORES es de (66179, 35)

--> Las columnas de CONDUCTORES son: 
 
['IdConductor', 'Sexo', 'PortaLicencia', 'CodigoCategoriaLicencia', 'CodigoRestriccionLicencia', 'ModeloVehiculo', 'CapacidadCarga', 'CantidadPasajeros', 'ClaseVehiculo', 'ServicioVehiculo', 'PosseSeguroResponsabilidad', 'CON_BICICLETA', 'CON_CARGA', 'CON_EMBRIAGUEZ', 'CON_HUECOS', 'CON_MENORES', 'CON_MOTO', 'CON_PEATON', 'CON_PERSONA_MAYOR', 'CON_RUTAS', 'CON_TPI', 'CON_VELOCIDAD', 'Edad', 'Gravedad', 'VehiculoViajabaClasificado', 'HuboFallaVehiculo', 'CumpleNormasSeguridad', 'CantidadVictimas', 'Cantidad_HERIDO_VALORADO', 'Cantidad_HERIDO_HOSPITALIZADO', 'Cantidad_MUERTA', 'Cantidad_PEATON', 'Cantidad_ACOMPANANTE', 'Cant

       VehiculoViajabaClasificado CON_BICICLETA CON_CARGA CON_EMBRIAGUEZ  \
count                       66179         66179     66179          66179   
unique                          7             2         2              2   
top                       LIVIANO            NO        NO             NO   
freq                        31678         60958     56296          65052   

       CON_RUTAS       Sexo CON_VELOCIDAD  
count      66179      66179         66179  
unique         2          3             2  
top           NO  MASCULINO            NO  
freq       66158      57081         65788  

Empty DataFrame
Columns: []
Index: [count, unique, top, freq]




In [15]:
CONDUCTORES.to_excel("data/CONDUCTORES_AGG.xlsx")